In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
import itertools
from tqdm import tqdm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score


warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)
warnings.simplefilter("ignore", category=UserWarning)

In [10]:
df

,Unnamed: 0,key,ticker,date,market_cap,type,%c1m_screen,%c2m_screen,%c3m_screen,%c4m_screen,%c5m_screen,%c10m_screen,%c15m_screen,%c20m_screen,%c25m_screen,%c30m_screen,%c45m_screen,%c60m_screen,%c75m_screen,%c90m_screen,%c105m_screen,%c120m_screen,%c135m_screen,%c150m_screen,%c165m_screen,%c180m_screen,%c195m_screen,%c210m_screen,%c225m_screen,%c240m_screen,%c255m_screen,%c270m_screen,%c285m_screen,%c300m_screen,%c315m_screen,%c330m_screen,%c345m_screen,%c360m_screen,%c375m_screen,%c1m,%c2m,%c3m,%c4m,%c5m,%c10m,%c15m,%c20m,%c25m,%c30m,%c45m,%c60m,%c75m,%c90m,%c105m,%c120m,%c135m,%c150m,%c165m,%c180m,%c195m,%c210m,%c225m,%c240m,%c255m,%c270m,%c285m,%c300m,%c315m,%c330m,%c345m,%c360m,%c375m,%cintra,%c1d,%cpah,%c2d,%c3d,%c1m_lag1,%c1m_lag2,%c1m_lag3,%c2m_lag1,%c2m_lag2,%c2m_lag3,%c3m_lag1,%c3m_lag2,%c3m_lag3,%c4m_lag1,%c4m_lag2,%c4m_lag3,%c5m_lag1,%c5m_lag2,%c5m_lag3,%c10m_lag1,%c10m_lag2,%c10m_lag3,%c15m_lag1,%c15m_lag2,%c15m_lag3,%c20m_lag1,%c20m_lag2,%c20m_lag3,%c25m_lag1,%c25m_lag2,%c25m_lag3,%c30m_lag1,%c30m_lag2,%c30m_lag3,%c45m_lag1,%c45m_lag2,%c45m_lag3,%c60m_lag1,%c60m_lag2,%c60m_lag3,%c75m_lag1,%c75m_lag2,%c75m_lag3,%c90m_lag1,%c90m_lag2,%c90m_lag3,%c105m_lag1,%c105m_lag2,%c120m_lag1,%c120m_lag2,%c135m_lag1,%c150m_lag1,%c165m_lag1,%c180m_lag1,dv1,dv2,dv3,dv4,dv5,dv10,dv15,dv20,dv25,dv30,dv45,dv60,dv75,dv90,dv105,dv120,dv135,dv150,dv165,dv180,dv195,dv210,dv225,dv240,dv255,dv270,dv285,dv300,dv315,dv330,dv345,dv360,dv375,dv389,15:59_dv,9:30_dv,first_tick_time,close_quote_time,open_quote_time,vwap_return,vwap_ft_return,ask_bid_return,mid_return,vwap_sfill_return,SPY_ah,SPY_pc_day,SPY_pc_timed,VONE_ah,VONE_pc_day,VONE_pc_timed,IWM_ah,IWM_pc_day,IWM_pc_timed,VTHR_ah,VTHR_pc_day,VTHR_pc_timed,VTI_ah,VTI_pc_day,VTI_pc_timed,log_dv1,log_dv2,log_dv3,log_dv4,log_dv5,log_dv10,log_dv15,log_dv20,log_dv25,log_dv30,log_dv45,log_dv60,log_dv75,log_dv90,log_dv105,log_dv120,log_dv135,log_dv150,log_dv165,log_dv180,log_dv195,log_dv210,log_dv225,log_dv240,log_dv255,log_dv270,log_dv285,log_dv300,log_dv315,log_dv330,log_dv345,log_dv360,log_dv375,log_dv389,log_15:59_dv,log_9:30_dv
0,0,AA_2021-03-18,AA,2021-03-18,5.835747e+09,CS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0.001304,0.000257,-0.000240,0.000876,0.002279,0.005028,-0.001882,-0.005692,-0.009080,-0.017786,-0.027054,-0.034564,-0.026753,-0.036191,-0.043400,-0.054587,-0.052441,-0.049500,-0.048593,-0.048108,-0.064327,-0.062552,-0.063309,-0.064734,-0.062746,-0.056270,-0.057924,-0.053594,-0.051319,-0.047369,-0.028878,-0.025162,-0.014105,-0.015161,-0.019805,-0.004715,0.056233,0.004510,-0.001046,-0.000497,0.001117,0.000620,0.001675,0.001843,0.002794,0.002902,-0.002772,0.003521,-0.001717,-0.006073,0.002743,-0.006876,-0.003817,-0.010667,-0.012163,-0.004185,-0.015933,-0.009436,-0.007719,-0.016297,-0.012952,0.000353,-0.019314,0.001508,-0.011207,-0.017082,-0.001685,-0.019087,-0.009391,-0.016860,0.004573,-0.020740,0.006854,-0.017467,-0.023372,-0.014528,0.010372,-0.012364,-0.008575,0.032963,-0.020021,0.011982,-0.010732,0.042311,-0.004041,-0.004307,0.001286,0.024105,1.752712e+06,3.233943e+06,4.604353e+06,7.170900e+06,8.195136e+06,1.548011e+07,2.021786e+07,2.379555e+07,2.802238e+07,3.098273e+07,3.794163e+07,4.189486e+07,4.685407e+07,5.312496e+07,5.786741e+07,6.280956e+07,6.727368e+07,7.079690e+07,7.377625e+07,8.085689e+07,8.774379e+07,9.386434e+07,1.004121e+08,1.044643e+08,1.109343e+08,1.187554e+08,1.286101e+08,1.374654e+08,1.444339e+08,1.622406e+08,1.710017e+08,1.773235e+08,1.848873e+08,2.007916e+08,3.327471e+06,2.294951e+07,1.0,15:59,09:30,-0.000288,-0.002512,-0.002566,0.000481,-0.000288,-0.004087,-0.014550,-0.014223,0.000327,-0.015562,-0.016045,-0.002975,-0.030433,-0.030132,0.002294,-0.016897,-0.016897,0.000683,-0.018199,-0.017241,14.376675,14.989213,15.342513,15.785542,15.919051,16.555066,16.822077,16.985009,17.148514,17.248941,17.451559,17.550674,17.662549,17.788158,17.873665,17.955618,18.024280,18.075326,18.116547,18.208191,18.289932,18.357361,18.424793,18.464356,18.524449,1

In [12]:
df[df['%c75m_screen'] == 1]['vwap_sfill_return'].describe()

count    30056.000000
mean         0.010202
std          0.100471
min         -0.817728
25%         -0.016292
50%          0.006119
75%          0.029762
max          3.505784
Name: vwap_sfill_return, dtype: float64

In [9]:
df[(df['%c1d'] < -.05) & (df['%c60m'] < -0.08)]['vwap_sfill_return'].describe()

count    2557.000000
mean        0.032169
std         0.133029
min        -0.817728
25%        -0.014475
50%         0.019567
75%         0.059408
max         1.963574
Name: vwap_sfill_return, dtype: float64

In [3]:
df = pd.read_csv('main.csv')
pct_feats = [col for col in df.columns if '%' in col and 'lag' not in col and 'screen' not in col]
lag_feats = [col for col in df.columns if 'lag' in col]
screen = [col for col in df.columns if 'screen' in col]
dollar_volumes = [col for col in df.columns if 'dv' in col]
returns = ['vwap_return', 'vwap_ft_return', 'ask_bid_return', 'mid_return','vwap_sfill_return']
entry_desc = ['first_tick_time','close_quote_time','open_quote_time']
intra = [col for col in df.columns if 'pc' in col]
for col in dollar_volumes:
    # Create new columns with the log-transformed values
    df[f'log_{col}'] = np.log(df[col] + 1e-10)
dv_logs = [col for col in df.columns if 'log' in col]

In [4]:
means = []
medians = []
stds = []
counts = []
n = 35
combos = [list(combo) for combo in itertools.combinations(pct_feats, n)]
qt = .03
for cols in tqdm(combos,total=len(combos)):
    mean,median,std,count = df[df[cols].lt(df[cols].quantile(qt)).any(axis=1)]['vwap_sfill_return'].agg(['mean','median','std','count'])
    means.append(mean)
    medians.append(median)
    stds.append(std)
    counts.append(count)
inter = pd.DataFrame({'feats':combos,'mean':means,'median':medians,'std':stds,'count':counts}).sort_values(by='mean',ascending=False).dropna()
inter

100%|██████████| 8436/8436 [18:27<00:00,  7.62it/s]


,feats,mean,median,std,count
9,"[%c1m, %c2m, %c3m, %c4m, %c5m, %c10m, %c15m, %...",0.008981,0.003090,0.140896,31975.0
18,"[%c1m, %c2m, %c3m, %c4m, %c5m, %c10m, %c15m, %...",0.008802,0.003026,0.140044,32506.0
15,"[%c1m, %c2m, %c3m, %c4m, %c5m, %c10m, %c15m, %...",0.008669,0.002850,0.140508,32345.0
28,"[%c1m, %c2m, %c3m, %c4m, %c5m, %c10m, %c15m, %...",0.008665,0.002899,0.139862,32658.0
7148,"[%c1m, %c3m, %c4m, %c5m, %c10m, %c15m, %c20m, ...",0.008626,0.002920,0.140619,31890.0
...,...,...,...,...,...
8204,"[%c2m, %c3m, %c4m, %c5m, %c10m, %c15m, %c20m, ...",0.007355,0.002119,0.136938,32376.0
4467,"[%c1m, %c2m, %c3m, %c4m, %c5m, %c10m, %c15m, %...",0.007351,0.002097,0.137731,33965.0
2003,"[%c1m, %c2m, %c3m, %c4m, %c5m, %c10m, %c15m, %...",0.007344,0.002214,0.136552,34118.0
4032,"[%c1m, %c2m, %c3m, %c4m, %c5m, %c10m, %c15m, %...",0.007313,0.002052,0.136746,33475.0


In [5]:
means = []
medians = []
stds = []
counts = []
for col in pct_feats:
    for qt in [.03]:
        mn,me,std,ct = df[df[col] < df[col].quantile(qt)]['vwap_sfill_return'].agg(['mean','median','std','count'])
        means.append(mn)
        medians.append(me)
        stds.append(std)
        counts.append(ct)
sing = pd.DataFrame({'col':pct_feats,'mean':means,'median':medians,'std':stds,'count':counts}).sort_values(by='mean',ascending=False)

In [6]:
means = []
medians = []
stds = []
counts = []
use = []
qt = .02
for x in range(1,39):
    cols = sing['col'].head(x).to_list()
    mean,median,std,count = df[df[cols].lt(df[cols].quantile(qt)).any(axis=1)]['vwap_sfill_return'].agg(['mean','median','std','count'])
    means.append(mean)
    medians.append(median)
    stds.append(std)
    counts.append(count)
    use.append(cols)
inter = pd.DataFrame({'feats':use,'mean':means,'median':medians,'std':stds,'count':counts})
inter

,feats,mean,median,std,count
0,[%c120m],0.029171,0.014469,0.174461,3168.0
1,"[%c120m, %c75m]",0.026073,0.011834,0.176919,4486.0
2,"[%c120m, %c75m, %c135m]",0.024542,0.011400,0.172555,5012.0
3,"[%c120m, %c75m, %c135m, %c105m]",0.023472,0.010861,0.170528,5348.0
4,"[%c120m, %c75m, %c135m, %c105m, %c90m]",0.022868,0.010318,0.172206,5612.0
5,"[%c120m, %c75m, %c135m, %c105m, %c90m, %c45m]",0.022587,0.009254,0.177102,6585.0
6,"[%c120m, %c75m, %c135m, %c105m, %c90m, %c45m, ...",0.021585,0.008347,0.179923,7823.0
7,"[%c120m, %c75m, %c135m, %c105m, %c90m, %c45m, ...",0.021204,0.008146,0.182889,8226.0
8,"[%c120m, %c75m, %c135m, %c105m, %c90m, %c45m, ...",0.020775,0.007729,0.183595,8590.0
9,"[%c120m, %c75m, %c135m, %c105m, %c90m, %c45m, ...",0.020308,0.007234,0.183322,8812.0


In [14]:
sing

,col,mean,median,std,count
15,%c120m,0.024191,0.012439,0.164681,4751.0
12,%c75m,0.024138,0.012096,0.166777,4774.0
16,%c135m,0.024127,0.012219,0.158780,4743.0
14,%c105m,0.024048,0.012903,0.159127,4757.0
13,%c90m,0.023694,0.012128,0.163733,4764.0
10,%c45m,0.023233,0.010759,0.166455,4769.0
7,%c20m,0.023167,0.011085,0.176643,4725.0
9,%c30m,0.022742,0.010229,0.177574,4750.0
17,%c150m,0.022661,0.012208,0.160120,4740.0
8,%c25m,0.021634,0.009563,0.177018,4736.0
